# Loading the Dataset

In [5]:
from dataset.dataset_loader import dataset_loader

train, test = dataset_loader((224, 224), data_folder_path="../data/")


Is the dataset balanced?

In [6]:
import numpy as np

train_chihuahuas = np.sum(train.targets)
test_chihuahuas = np.sum(test.targets)

In [7]:
print(f"We have a total of {train_chihuahuas} and {len(train) - train_chihuahuas} muffins in the training set. "
      f"This results for a ratio of {train_chihuahuas / len(train) * 100:.2f}% of chihuahuas .")
print(f"We have a total of {test_chihuahuas} and {len(test) - test_chihuahuas} muffins in the testing set."
      f"This results for a ratio of {test_chihuahuas / len(test) * 100:.2f}% of chihuahuas .")

We have a total of 2174 and 2559 muffins in the training set. This results for a ratio of 45.93% of chihuahuas .
We have a total of 544 and 640 muffins in the testing set.This results for a ratio of 45.95% of chihuahuas .


In [15]:
print(
    f"We have a total of: {len(train) + len(test)} samples, of which {len(test) / (len(train) + len(test)) * 100:.2f}% is in the test set ")

We have a total of: 5917 samples, of which 20.01% is in the test set 


# Does KFold Split evenly?
Does my implementation of the K-Fold Cross Validation split the dataset in a desirable way?
(Meaning it is correct)

In [ ]:
from dataset.k_fold_dataset_wrapper import KFoldDatasetWrapper as KFoldController

dataset_split_controller = KFoldController(5)
dataset_split_controller.load_data(train)

local_train, validation = dataset_split_controller.get_data_for_fold(0)

k_fold_train = np.sum([local_train[i][1] for i in range(len(local_train))])
k_fold_test = np.sum([validation[i][1] for i in range(len(validation))])

In [ ]:
print(f"We have a total of {k_fold_train} and {len(local_train) - k_fold_train} muffins in the training set. "
      f"This results for a ratio of {k_fold_train / len(local_train) * 100:.2f}% of chihuahuas .")
print(f"We have a total of {k_fold_test} and {len(validation) - k_fold_test} muffins in the testing set."
      f"This results for a ratio of {k_fold_test / len(validation) * 100:.2f}% of chihuahuas .")

# Normalizing the dataset
Normalizing the data has the advantage of speeding up the training.

In [2]:
from dataset.dataset_loader import dataset_loader

# Load the datasets to measure the mean and std
train, test = dataset_loader((224, 224), data_folder_path="../data/")

>  Common pitfall. An important point to make about the preprocessing is that any preprocessing statistics
     (e.g. the data mean) must only be computed on the training data, and then applied to the validation / test data.
     E.g. computing the mean and subtracting it from every image across the entire dataset and then splitting
     the data into train/val/test splits would be a mistake. Instead, the mean must be computed only over the
     training data and then subtracted equally from all splits (train/val/test).
>     
>              ~ https://cs231n.github.io/neural-networks-2/ (Data Preprocessing)


Because of this pitfall when doing K-fold CV or diving the dataset in train/validation we will have to compute the values again and pass them to the model in an augmentation procedure.

In [6]:
from dataset_loader import dataset_information

res = dataset_information(train, (224, 224))

In [9]:
print(f"The training dataset has means: {res[0]} and stds: {res[1]} (w.r.t. the channels)")

The training dataset has means: tensor([0.6501, 0.5935, 0.5400]) and stds: tensor([0.2951, 0.3003, 0.3205]) (w.r.t. the channels)


In [74]:
import torch
import numpy as np
import plotly.express as px
from torch.utils.data import DataLoader
import keras

x = keras.layers.Normalization(axis=1, mean=[0.6501, 0.5935, 0.5400], variance=[0.2951, 0.3003, 0.3205])
train_dataloader = DataLoader(dataset=train, batch_size=16, shuffle=True)

permute = keras.layers.Permute(dims=(2, 3, 1))

current_image = train[22][0].unsqueeze(0)
normalized_image = x(current_image)

print(permute(normalized_image).cpu().shape)
print(permute(current_image).cpu().shape)

list = [
    permute(normalized_image).squeeze(0).cpu(),
    permute(current_image).squeeze(0).cpu()
]



image_show = px.imshow(np.array(list), binary_string=True, facet_col=0, facet_col_wrap=4)
image_show.show()

torch.Size([1, 224, 224, 3])
torch.Size([1, 224, 224, 3])


These values are stored in the dataset_loader.py file as a dictionary.

## Augmentation?

In [ ]:
#TBDtest_chihuahuas